In [1]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter Notebooks

In [7]:
# Load the main dataset
df = pd.read_csv("data/java_test_dataset.csv")

- consistent downloading process and stable connection, retry if it gets down
- process stage representation and showing in terminal
- thread for faster internet connection and communication 
- error handeling and pass if a file or repository dosnt exist (try main instead of master if a file content doesnt count). 

In [9]:
def get_requests_session():
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
    session.mount('http://', HTTPAdapter(max_retries=retries))
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def get_raw_content_url(github_url, file_path, branch='master'):
    parts = github_url.split('/')
    user, repo = parts[3], parts[4]
    raw_url = f'https://raw.githubusercontent.com/{user}/{repo}/{branch}/{file_path}'
    return raw_url

def download_file_content(args):
    url, focal_file_path, test_file_path, session = args
    results = {}
    for file_type, file_path in [('focal_class_code', focal_file_path), ('test_class_code', test_file_path)]:
        for branch in ['master', 'main']:  # Try with 'master' first, then 'main'
            raw_url = get_raw_content_url(url, file_path, branch)
            try:
                response = session.get(raw_url)
                response.raise_for_status()
                results[file_type] = response.text
                break  # Break if successfully got the content
            except requests.RequestException:
                pass  # Try the next branch
        if file_type not in results:  # If both attempts fail
            results[file_type] = None
    return results

def download_files_concurrently(df):
    session = get_requests_session()
    with ThreadPoolExecutor(max_workers=50) as executor:
        # Update to include both focal and test class file paths
        tasks = [(row['url'], row['focal_class_file'], row['test_class_file'], session) for index, row in df.iterrows()]
        futures = {executor.submit(download_file_content, task): index for index, task in enumerate(tasks)}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading"):
            result = future.result()
            index = futures[future]
            # Assign the results to the new columns
            df.at[index, 'focal_class_code'] = result['focal_class_code']
            df.at[index, 'test_class_code'] = result['test_class_code']


In [10]:
# The new columns initialized
df['focal_class_code'] = None
df['test_class_code'] = None

In [11]:
# Perform the downloading and updating
download_files_concurrently(df)

In [12]:
df.head()

,repo_id,url,language,fork_count,stargazer_count,focal_class_identifier,focal_class_superclass,focal_class_interfaces,focal_class_fields,focal_class_methods,...,test_case_return,test_case_body,test_case_signature,test_case_full_signature,test_case_class_method_signature,test_case_testcase,test_case_constructor,test_case_invocations,focal_class_code,test_class_code
0,58314354,https://github.com/bytefish/JavaElasticSearchE...,Java,11,0,IgnoreMissingValuesConverter,NaN,implements ITypeConverter<Float>,[{'original_string': 'private List<String> mis...,[{'identifier': 'IgnoreMissingValuesConverter'...,...,void,@Test\n public void returns_null_if_value_i...,void returns_null_if_value_is_missing(),@Test public void returns_null_if_value_is_mis...,IgnoreMissingValuesConverterTest.returns_null_...,True,False,"['assertEquals', 'convert', 'assertEquals', 'c...",// Copyright (c) Philipp Wagner. All rights re...,// Copyright (c) Philipp Wagner. All rights re...
1,58314354,https://github.com/bytefish/JavaElasticSearchE...,Java,11,0,DateUtilities,NaN,NaN,[],"[{'identifier': 'from', 'parameters': '(LocalD...",...,void,@Test\n public void generated_date_has_utc_...,void generated_date_has_utc_offset_when_none_i...,@Test public void generated_date_has_utc_offse...,DateUtilitiesTest.generated_date_has_utc_offse...,True,False,"['of', 'of', 'atTime', 'atOffset', 'from', 'as...",// Copyright (c) Philipp Wagner. All rights re...,// Copyright (c) Philipp Wagner. All rights re...
2,58314354,https://github.com/bytefish/JavaElasticSearchE...,Java,11,0,DateUtilities,NaN,NaN,[],"[{'identifier': 'from', 'parameters': '(LocalD...",...,void,@Test\n public void generated_date_has_give...,void generated_date_has_given_offset_when_offs...,@Test public void generated_date_has_given_off...,DateUtilitiesTest.generated_date_has_given_off...,True,False,"['of', 'of', 'ofHours', 'atTime', 'atOffset', ...",// Copyright (c) Philipp Wagner. All rights re...,// Copyright (c) Philipp Wagner. All rights re...
3,58314354,https://github.com/bytefish/JavaElasticSearchE...,Java,11,0,DateUtilities,NaN,NaN,[],"[{'identifier': 'from', 'parameters': '(LocalD...",...,void,@Test\n public void generated_date_has_give...,void generated_date_has_given_timezone_when_gi...,@Test public void generated_date_has_given_tim...,DateUtilitiesTest.generated_date_has_given_tim...,True,False,"['of', 'of', 'atTime', 'ofHours', 'atOffset', ...",// Copyright (c) Philipp Wagner. All rights re...,// Copyright (c) Philipp Wagner. All rights re...
4,58314354,https://github.com/bytefish/JavaElasticSearchE...,Java,11,0,LocalWeatherDataConverter,NaN,NaN,[],"[{'identifier': 'convert', 'parameters': '(csv...",...,void,@Test\n public void testConvert() throws Ex...,void testConvert(),@Test public void testConvert(),LocalWeatherDataConverterTest.testConvert(),True,False,"['setWban', 'setDate', 'of', 'setTime', 'of', ...",// Copyright (c) Philipp Wagner. All rights re...,// Copyright (c) Philipp Wagner. All rights re...


In [14]:
# print(df['focal_class_code'][100])
df.to_csv("data/java_test_dataset_code.csv", index=False)